In [5]:
# Set environment variables
import os
import pandas as pd

# Start a Spark session
import findspark
findspark.init()

# Create a SparkSession
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


spark = (
    SparkSession.builder.appName("MyApp")
    .config("spark.driver.memory", "12g") #Change based on your system's available memory
    .config("spark.executor.memory", "12g")
    .getOrCreate()
)
spark.sparkContext.setCheckpointDir("../data/spark_checkpoint")

os.chdir("../data/")

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [4]:
# Read parquet file to DF
folder1 = "../data/usps_0501.parquet"
folder2 = "../data/usps_0502.parquet"
folder3 = "../data/usps_0503.parquet"
parc_folders = [folder1,folder2,folder3]
df = spark.read.parquet(*parc_folders)

#Cleaning -----
# data from 0503 read ServiceType code as int so it removed 0s to the left;adding those back
df = df.withColumn('ServiceTypeCode', lpad(col('ServiceTypeCode'), 3, '0'))

# Get the list of columns that contain 'zipcode'
zipcode_columns = [col for col in df.columns if 'zipcode' in col]

# Define a function to add leading zeros
def add_leading_zeros(col):
    return lpad(col, 5, '0')

# Apply the function to all zipcode columns
for col in zipcode_columns:
    df = df.withColumn(col, add_leading_zeros(df[col]))

# recode CMRA flag
df = df.withColumn(
    'CRID_cmra_flag',
    when(df['CRID_cmra_flag'] == 'Y', True)
    .when(df['CRID_cmra_flag'].isin(['N', 'X']), False)
    .otherwise(None)
)

# recode delinked nulls
df = df.withColumn("delinked", when(df["delinked"].isNull(), False).otherwise(df["delinked"]))



NameError: name 'spark' is not defined

In [3]:
# create potential fraud column
df = df.withColumn(
    "potential_fraud",
    when(df["delinked"] == True, True)
    .when(df["icr_or_manifest"] == False, True)
    .when(df["symbology_mismatch"] == True, True)
    .when(df["firstzip_startzip_diff"] == True, True)
    .otherwise(False)
)

In [4]:
data = pd.read_excel("../data/intercept_list_MIDs_cleansed.xlsx")
df2_spark = spark.createDataFrame(data)

In [5]:
df1 = df.withColumn("intercepted_fraud", lit(False))

In [6]:
df_joined = df1.join(df2_spark, df1.mailerid == df2_spark.mailerid, how="leftsemi").withColumn("temp", lit(True))
df_joined = df_joined.drop("intercepted_fraud").withColumnRenamed("temp", "intercepted_fraud")
df_antijoin = df1.join(df2_spark, df1.mailerid == df2_spark.mailerid, how="leftanti")

In [7]:
df_labeled = df_joined.union(df_antijoin)

In [8]:
df_labeled = df_labeled.coalesce(1)
df_labeled.write.csv("../data/rawlabeled.csv")

24/06/21 05:03:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
ERROR:root:Exception while sending command. 0) / 16][Stage 3:>  (0 + 0) / 234]6]
Traceback (most recent call last):
  File "/home/supercliche/bbcudas_usps/.venv/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/supercliche/bbcudas_usps/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/supercliche/bbcudas_usps/.venv/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNe

Py4JError: An error occurred while calling o129.csv

In [ ]:
df_labeled_10_perc_subset = df_labeled.sample(fraction=0.1)
df_labeled_10_perc_subset = df_labeled_10_perc_subset.coalesce(1)
df_labeled_10_perc_subset.write.csv("../data/subset_10_perc_labeled.csv")

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
df_labeled_1_perc_subset = df_labeled.sample(fraction=0.01)
df_labeled_1_perc_subset = df_labeled_1_perc_subset.coalesce(1)
df_labeled_1_perc_subset.write.csv("../data/subset_1_perc_labeled.csv")